<a href="https://colab.research.google.com/github/ashinkrishnan/FakeNewsClassifier-Using-BidirectionalLSTM.ipynb/blob/main/FakeNewsClassifier_Using_BidirectionalLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/fake_news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df = df.dropna()
X = df.drop('label',axis=1)
y = df['label']

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout


In [4]:
#Vocabulary size
voc_size = 5000

In [5]:
messages = X.copy()
messages.reset_index(inplace=True)
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [6]:

import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i,end='\r')
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [8]:
corpus[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [9]:
#onehot representation
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[:5]

[[1405, 1726, 4857, 4431, 885, 4237, 1317, 1204, 2676, 740],
 [2530, 37, 1006, 3800, 1589, 1303, 4370],
 [2919, 2211, 681, 4257],
 [4430, 2170, 800, 1753, 1608, 4555],
 [4063, 1589, 1115, 3285, 2243, 4649, 1589, 3240, 150, 29]]

In [10]:
#Embedding reapresentation
sent_length=20
embedded_docs = pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
embedded_docs[:5]

array([[1405, 1726, 4857, 4431,  885, 4237, 1317, 1204, 2676,  740,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [2530,   37, 1006, 3800, 1589, 1303, 4370,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [2919, 2211,  681, 4257,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [4430, 2170,  800, 1753, 1608, 4555,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [4063, 1589, 1115, 3285, 2243, 4649, 1589, 3240,  150,   29,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

In [11]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [14]:
#model training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 14ms/step - loss: 0.3097 - accuracy: 0.8518 - val_loss: 0.1933 - val_accuracy: 0.9175
Epoch 2/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1234 - accuracy: 0.9533 - val_loss: 0.2035 - val_accuracy: 0.9186
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0793 - accuracy: 0.9731 - val_loss: 0.2527 - val_accuracy: 0.9153
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0529 - accuracy: 0.9826 - val_loss: 0.3187 - val_accuracy: 0.9152
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0371 - accuracy: 0.9867 - val_loss: 0.3486 - val_accuracy: 0.9062
Epoch 6/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0268 - accuracy: 0.9918 - val_loss: 0.3626 - val_accuracy: 0.9062
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0165 - accuracy: 0.9958 - val_loss: 0.3850 - val_accuracy: 0.905

In [15]:
y_pred = model1.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [20]:
#evaluation uisng confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3094,  325],
       [ 255, 2361]])

In [24]:
#evaluate the model accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.903893951946976

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      3419
           1       0.88      0.90      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

